# AIFFEL 대전 1기 Going Deeper CV Lecture5. Class Activation Map
## Date: APRIL 19, 2021

# 1. XAI(Explainable AI)
- 설명 가능한 인공지능

# 2. logit, sigmoid, softmax
![logit_sigmoid_softmax](https://s3-ap-northeast-2.amazonaws.com/opentutorials-user-file/module/3653/9840.png)  

1) logit <-> sigmoid: 역함수 관계  
2) logit(2개 클래스를 대상으로 정의됨)을 K개의 클래스를 대상으로 일반화하면 softmax함수 유도  
3) softmax함수에서 K를 2로 설정 -> sigmoid함수 환원 (반대로 sigmoid함수를 K개의 클래스를 대상으로 일반화 -> softmax)  

> 출처: https://opentutorials.org/module/3653/22995  

logit은 아래와 같이 sigmoid와 역함수  
$t = logit(y) = \frac{y}{1-y}$  

softmax는 sigmoid를 K개의 클래스로 일반화  
$y = sigmoid(t) = \frac{1}{1+exp(t)}$

# 3. Class Acitvation Map
- "모델이 어떤 곳을 보고 각 클래스를 짐작하고 있는지"확인할 수 있는 지도.  

## 1) GAP(Global Average Pooling)
Network in Network 논문에서는 CNN 이후 fully connected layer를 사용하는 대신 아래 그림처럼 GAP 연산을 한 후 이에 대해 소프트맥스 활성화 함수를 적용합니다. 이때 마지막 CNN 레이어의 채널 수는 데이터의 클래스 수에 맞춰 각 클래스에 따른 확률을 얻을 수 있도록 했습니다. 원 논문에서는 이렇게 하면 특성 맵의 각 채널이 클래스별 신뢰도를 나타내게 되어 해석이 쉬울 뿐만 아니라, fully connected layer과 달리 최적화할 파라미터가 존재하지 않으므로 과적합(overfitting)을 방지할 수 있다고 설명합니다.  
![p](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-3-L-3.max-800x600.png)  

## 2) CAM

![dd](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-3-L-5.max-800x600.png)  

CNN 레이어를 거친 특성 맵에도 입력값의 위치정보가 유지.  
이 특성 맵에 GAP를 적용.  
그리고 이에 대해서 소프트맥스 적용(bias가 없는 FCN사용)  
CNN을 거친 특성맵에서 각 클래스에 대한 정보는 결과값의 여러 채널에 걸쳐 나타나게 됩니다. GAP을 통해 각 채널 별 정보를 요약하면 소프트맥스 레이어는 이 정보를 보고 각 클래스에 대한 개별 채널의 중요도를 결정  
이렇게 클래스 별로 소프트맥스 레이어를 통해 각 채널의 가중합을 구하면 각 클래스가 활성화 맵의 어떤 부분을 주로 활성화 시키는지 확인 가능.  
이렇게 얻은 특성 맵은 CNN의 출력값 크기와 같다.  
이를 보간(interpolation)을 통해 적절히 원본 이미지 크기로 확대해 주면 위와 같은 CAM을 얻을 수 있다.

# 4. GradCAM
- CNN 기반의 네트워크는 굳이 **모델 구조를 변경할 필요가 없으며, 분류 문제 외의 다른 태스크들에 유연하게 대처** 가능  
CAM의 경우에는 활성화 맵을 얻기 위해서 GAP을 사용하여 $f_k$를 구하고, 그 뒤로 fully connected layer $w^c_k$를 추가로 붙여야 함.  
![gradcam](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-3-L-8.max-800x600.png)  

위의 이미지는 개와 고양이가 있는 이미지에 대해서 Guided Backprop, Grad-CAM, Occlussion map의 시각화를 비교하고 있습니다. Grad-CAM에서는 높은 분별력과 큰 dimension을 갖는 CAM을 만드는 것을 중요하게 보았다.  

![gc](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-3-L-9.max-800x600.png)  
- Grad-CAM의 전체적인 구조.  
- Visual question answering은 VQA라고도 불립니다. 어떤 질문과 이미지가 주어졌을 때 이에 대한 답변을 내는 태스크  
- Grad-CAM은 이런 복잡한 태스크에 사용되는 모델에서도 사용될 수 있다는 점이 장점

# 5. ACoL: Adversarial Complementary Learning
- 약지도학습(Weakly Supervised Learning): 직접적으로 정답 위치정보를 주지 않아도 간접적인 정보를 활용하여 학습을 하고 원하는 정보를 얻어낼 수 있도록 모델을 학습하는 방식. CAM, Grad-CAM 그리고 지금 설명드리는 ACoL은 약지도학습 기법을 활용한 물체 검출(object detection)을 수행 가능.  

# <약지도 학습 용어 정리>

## 1) incomplete supervison
-  학습 데이터 중 일부에만 라벨이 달려 있는 경우 (예: 개와 고양이 분류 학습시 10000개의 이미지 중 1000개만 라벨이 있는 경우) 이 경우가 일반적으로 말하는 준지도학습과 같은 경우  

## 2) inexact supervision
- 학습데이터의 라벨이 충분히 정확하게 달려있지 않은 경우. (예: 개나 고양이를 Object Detection 또는 Semantic Segmentation해야 하는데, 이미지 내에 정확한 bounding box는 주어져 있지 않고 이미지가 개인지 고양인지 정보만 라벨로 달려있는 경우)  

## 3) inaccurate supervision
-  학습 데이터에 Noise가 있는 경우 (예: 개나 고양이의 라벨이 잘못 달려있는 경우)   

일반적으로 Image Classification용 학습데이터보다는 bounding box 정보까지 정확하게 포함해야 하는 Object Detection이나 Semantic Segmentation을 위한 학습데이터가 훨씬 제작비용이 많이 든다.  
아래는 Grad-CAM 논문에서 언급한 약지도학습을 통한 Object Detection과 Semantic Segmentation의 예시(Grad-CAM을 통한 Counterfactual Explanation 예시)    
![catdog](https://aiffelstaticprd.blob.core.windows.net/media/original_images/grad-cam-counterfactual.png)  
1. Grad-CAM을 통해 개와 고양이의 특징이 두드러지게 하는 영역의 가중치 점수를 계산  
2. 오히려 해당 가중치 점수를 제거해 주면 Image classification 모델에서 해당 클래스에 대한 prediction이 바뀌게 될수도 있다.  
3. 그렇게 제거했을 때 prediction이 바뀌도록 하는 가중치 영역을 모으면 한번도 bounding box 라벨을 보지 않고서도 object detection을 해낼 수 있다.  

아래는 Grad-CAM을 이용해서, 위와 비슷한 논리로 Semantic Segmentation도 수행 가능함을 보여줌.  
![g](https://aiffelstaticprd.blob.core.windows.net/media/original_images/grad-cam_segmentation.png)  

## ACoL
- Adversarial의 뜻은 "적대적인"  
- CAM, Grad-CAM의 문제점: 물체의 가장자리보다 **특징이 주로 나타나는 위치에 중점적으로 활성화**된다. 전체적인 윤곽을 잡아내는 것이 더 중요.  
- CAM 모델이 특정 부위에 집중해 학습하는 것을 막기 위해 ACoL은 **브랜치를 두 가지로 두어 너무 높은 점수를 지워줌**으로서 주변의 특성 또한 반영하도록 했다.  

![acol](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-3-L-13.max-800x600.png)  
먼저 위쪽 주황색 브랜치를 거칩니다. 특성 맵은 GAP을 거쳐 CAM에서 보았던 소프트맥스 레이어인 `Classifier A`를 거치게 됩니다. 이 브랜치는 loss로 학습됩니다. ACoL은 여기서 얻어진 활성화 맵을 적대적인 방법으로 사용합니다. 일정 값 이상 활성화된 활성화 맵을 지우도록 하는 것이지요. `Classifier A`는 쉽게 전체적인 이미지를 보고 클래스를 판별할 수 있는 반면에 `Classifier B`는 A의 CAM에서 크게 활성화된 영역을 지운 활성화 맵에서 분류를 해야하기 때문에 더 어려운 문제를 푸는 것으로 볼 수 있습니다. 이렇게 두 가지 `Classifier A`와 `Classifier B`를 학습시킴으로써 더 넓은 영역을 판별의 근거로 삼을 수 있습니다. 이 과정을 통해서 모델은 쉽게 맞출 수 있는 샘플을 어렵게 다시 한 번 학습을 하는 Adversarial Complementary Learning 과정을 거치게 됩니다.  

결과적으로 위 그림에서는 기존 CAM이 활성화 되는 부분과 달리 ACoL은 물체의 전반적인 영역으로 CAM이 활성화되는 효과를 확인할 수 있습니다.

CAM을 활용하면 멋진 프로젝트들을 만들 수 있습니다. 예컨대 황준식님의 [CAM: 대선주자 얼굴 위치 추적기](https://jsideas.net/class_activation_map/)에서는 2017년 대선주자의 얼굴을 분류하고 이 모델을 기반으로 어느 위치에 얼굴이 있는지를 찾는 실험까지 구현합니다.